In [ ]:
import os
import re
import random
import time

import pandas as pd
import numpy as np

import gensim
import nltk
from nltk.corpus import stopwords

import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [ ]:
import pickle

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Data preparation

### Upload data

In [ ]:
os.listdir("/content/drive/MyDrive/comments")

['JoeBiden_comments_2020_all.csv',
 'The_Donald_comments_2020_hourly.csv',
 'Trump_comments_2020_all.csv',
 'Trump_comments_2020_clean.csv',
 'JoeBiden_comments_2020_clean.csv']

In [ ]:
#comments_dir = "Lecture - Text embeddings (Medvedev)/codes/comments"
comments_dir = "/content/drive/MyDrive/comments"

In [ ]:
# TRUMP COMMENTS
trump_comments_file = "./Trump_comments_2020_clean.csv"
trump_comments_path = os.path.join(comments_dir, trump_comments_file)
trump_df = pd.read_csv(trump_comments_path, index_col = 0)


# BIDEN COMMENTS
biden_comments_file = './JoeBiden_comments_2020_clean.csv'
biden_comments_path = os.path.join(comments_dir, biden_comments_file)
biden_df = pd.read_csv(biden_comments_path, index_col = 0)


In [ ]:
trump_df['who'] = 0
biden_df['who'] = 1

In [ ]:
all_df = pd.concat([trump_df, biden_df])
all_df = all_df.reset_index(drop=True)
all_df = all_df.drop(['author', 'created_utc', 'link_id', 'parent_id', 'score'], axis=1)

In [ ]:
len(trump_df), len(biden_df)

(618858, 499528)

In [ ]:
biden_df = biden_df[:20000]
trump_df = trump_df[:20000]
trump_df['who'] = 0 # 0 == trump
biden_df['who'] = 1 # 1 == biden
all_df = pd.concat([trump_df, biden_df])
all_df = all_df.reset_index(drop=True)
all_df["body"] = all_df["body"].apply(str)


In [ ]:
del(trump_df)
del(biden_df)

# Vectorization

In [ ]:
!pip install tensorflow==1.15
!pip install "tensorflow_hub>=0.6.0"
!pip3 install tensorflow_text==1.15

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub

In [ ]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder/2" #@param ["https://tfhub.dev/google/universal-sentence-encoder/2", "https://tfhub.dev/google/universal-sentence-encoder-large/3"]

In [ ]:
# Import the Universal Sentence Encoder's TF Hub module
embed = hub.Module(module_url)

# Compute a representation for each message, showing various lengths supported.
word = "Elephant"
sentence = "I am a sentence for which I would like to get its embedding."
paragraph = (
    "Universal Sentence Encoder embeddings also support short paragraphs. "
    "There is no hard limit on how long the paragraph is. Roughly, the longer "
    "the more 'diluted' the embedding will be.")
messages = [word, sentence, paragraph]

with tf.Session() as session:
  session.run([tf.global_variables_initializer(), tf.tables_initializer()])
  message_embeddings = session.run(embed(messages))

  for i, message_embedding in enumerate(np.array(message_embeddings).tolist()):
    print("Message: {}".format(messages[i]))
    print("Embedding size: {}".format(len(message_embedding)))
    message_embedding_snippet = ", ".join(
        (str(x) for x in message_embedding[:3]))
    print("Embedding: [{}, ...]\n".format(message_embedding_snippet))

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Message: Elephant
Embedding size: 512
Embedding: [-0.016987258568406105, -0.008949847891926765, -0.007062715478241444, ...]

Message: I am a sentence for which I would like to get its embedding.
Embedding size: 512
Embedding: [0.03531331941485405, -0.025384265929460526, -0.007880021817982197, ...]

Message: Universal Sentence Encoder embeddings also support short paragraphs. There is no hard limit on how long the paragraph is. Roughly, the longer the more 'diluted' the embedding will be.
Embedding size: 512
Embedding: [0.018790993839502335, 0.04536517709493637, -0.02001088112592697, ...]



In [ ]:
vecs = list()
with tf.Session() as session:
  session.run([tf.global_variables_initializer(), tf.tables_initializer()])
  step = 5000
  for i in range(step, len(all_df)+step, step):
    print(i)
    tmp_vecs = session.run(embed(list(all_df["body"][i-step:i])))
    vecs.append(tmp_vecs)

5000
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


10000
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


15000
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


...

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


35000
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


40000
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [ ]:
vecs = [item for sublist in vecs for item in sublist]

In [ ]:
len(vecs)

40000

In [ ]:
assert(len(vecs) == len(all_df))

In [ ]:
all_df["vec"] = vecs

In [ ]:
all_df.head()

,author,body,created_utc,link_id,parent_id,score,who,vec
0,khalabrakis,"Fighters... That means warrior, like people wh...",1577836803,t3_ei6h2x,t3_ei6h2x,1,0,"[0.045877255, -0.0048621236, -0.023708122, 0.0..."
1,Whopper_Jr,it is plausible that Republicans will never wi...,1577836806,t3_ei4ag7,t3_ei4ag7,1,0,"[0.0047475738, 0.031999968, -0.029850133, -0.0..."
2,Sustainable_Saltmine,we must send bartenders back to bartending and...,1577836807,t3_eiay7b,t3_eiay7b,1,0,"[-0.009444212, 0.005957485, -0.00456313, 0.003..."
3,TheC0zmo,Bullshit. you are blaming a fiscally conservat...,1577836814,t3_ei40gl,t1_fco5fri,1,0,"[0.048155, 0.035600815, -0.031101977, 0.062596..."
4,brotherjustincrowe,"Military R&amp;D and intel is not ""no reason.""...",1577836818,t3_ei51lq,t1_fcodqek,1,0,"[-0.018903375, -0.044530015, -0.021175865, -0...."


In [ ]:
all_df = all_df.drop(columns=["author", "created_utc", "link_id", 	"parent_id", 	"score"])

In [ ]:
pickle.dump(all_df, open("df_use.pickle", 'wb'))

In [ ]:
from scipy.spatial.distance import cosine

In [ ]:
def cosine_similarity(a,b):
  return (1 - cosine(a,b))

In [ ]:
# trump_sim2 = []
# biden_sim2 = []
# tb_sim2 = []

# sample_value = 1000

# for i in range(sample_value):
#   trump_sample_1 = all_df.loc[all_df['who'] == 1]["vec"].sample().values[0]
#   trump_sample_2 = all_df.loc[all_df['who'] == 1]["vec"].sample().values[0]
#   trump_sim2.append(cosine_similarity(trump_sample_1, trump_sample_2))

#   biden_sample_1 = all_df.loc[all_df['who'] == 0]["vec"].sample().values[0]
#   biden_sample_2 = all_df.loc[all_df['who'] == 0]["vec"].sample().values[0]
#   biden_sim2.append(cosine_similarity(biden_sample_1, biden_sample_2))

#   tb_sim2.append(cosine_similarity(biden_sample_1, trump_sample_1))

In [ ]:
# plt.hist(trump_sim2, label='trump')
# plt.hist(biden_sim2, label='biden')
# plt.hist(tb_sim2, label='trump & biden')
# plt.legend()

# Classifier

In [ ]:
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn import utils
from sklearn.metrics import classification_report

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(all_df["vec"].tolist(), all_df["who"].tolist())


logreg = LogisticRegression(n_jobs=1, C=1e5)
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.84      0.86      0.85      5049
           1       0.85      0.84      0.84      4951

    accuracy                           0.85     10000
   macro avg       0.85      0.85      0.85     10000
weighted avg       0.85      0.85      0.85     10000



/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [ ]:
filename = 'USE_model.pickle'
pickle.dump(logreg, open(filename, 'wb'))
 
# some time later...
 
# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(X_test, y_test)
print(result)

0.8478


In [ ]:
#!pip install umap-learn

In [ ]:
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['figure.dpi'] = 200 # 200 e.g. is really fine, but slower

In [ ]:
import umap

In [ ]:
reducer = umap.UMAP(n_neighbors=3, min_dist=0.001, metric='cosine')

In [ ]:
embedding = reducer.fit_transform(all_df["vec"].tolist())
embedding.shape

In [ ]:
plt.scatter(
    embedding[:, 0],
    embedding[:, 1],
    c=all_df["who"].tolist(), cmap='Spectral', s=10, alpha=0.1, edgecolors = 'face')
plt.gca().set_aspect('equal', 'datalim')
plt.title('UMAP projection of the dataset', fontsize=24)
plt.figure(figsize=(10,7))